# Part 1: The Ising Model

We'll start with the *classical* Ising model, a simple physical model of magnetism. As we add complexity to the model, we'll be able to represent more interesting physics more accurately. Hopefully, the complexity of our *code* will grow more slowly than the complexity of the physics.

## Introduction to the Model

The Ising model is a powerful but simple model of magnetism in materials. Small constituents, local magnetic moments, interact with each other to give "macroscopic" phenomena, like long range correlations we observe as ferromagnetism.

A magnetic moment is a local "unit" of magnetism. We say that if something (an atom, an electron, an ion) has *non-zero* magnetic moment then it "feels" magnetic fields and responds to them. The grains in a hard drive have non-zero magnetic moment, as do all the nuclei in a human being (that's how MRI works).

These local magnetic moments will add up (or not) to give the macroscopic moment you'd see in a compass needle. The magnetic moment of a big object, made of many little magnetic pieces, is just the sum of all their magnetic moments (we'll return to this later). If most of the magnetic moments are *positive*, the sum will be a large positive number, and the magnet points north. If most are *negative*, the sum will be a large negative number, and the magnet will point south. All the parts of a human have magnetic moment but they average out to pointing no direction at all.

As an analogy, imagine a group of people holding a rope trying to pull a 747. If all of them pull in the same direction, their small individual efforts may combine to enough force to move the jet. However, if everyone is too busy pulling in whatever direction they feel, the jet won't move at all. If the people are on slippery ground and end up flummoxed, prevented from working together, again they cannot generate enough force to move the airplane. We want to find out what combination of community-mindedness and dry dirt will let the people move/not move the airplane. 

Let's make this a little concrete. For the workshop, we'll work in *two dimensions* on a *square lattice*. This is both easy for us to visualize and easy to represent on a computer. We have a square crystal, where on each site in the crystal, the *spin* (local magnetic moment) can be *up* or *down*.

Here's a sample configuration, represented in many equivalent ways:

![lattice](pics/lattice.png)
![separatespins](pics/separatespins.svg)
![spinstogether](pics/spinstogether.svg)
![zerosandones](pics/zerosandones.svg)

Another model you might have seen built out of similar constituents is [Conway's Game of Life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life). If you're familiar with cellular automata, this stuff might be sounding familiar as well.

Physical systems want to minimize their energy. The Ising model is a set of energetic interactions which add up to give the total energy of the system, which we'll write Julia code to minimize. By simulating the model we can learn about what physics is favourable in different parameter regimes. Another way of saying this is:
- If the small constituent parts have *this sort* of interaction (a set of parameters), what will we observe *overall* (the macroscopic physics)?
- How far can we vary the interactions and keep the same physics (parameter regimes)?
- What will happen when we cross from one regime into another?

Here is the (classical) energy function, which we call a Hamiltonian:

$$ H = -\sum_{\langle i, j \rangle} \sigma_i \sigma_j $$

There are a couple things we should explain here:
  - $\langle i, j \rangle$ means "nearest neighbors", so look at how site 1 and site 2 interact, but not site 1 and site 8.
  - $\sigma_i$ is the local configuration (the arrow) on site $i$. It can be 0 (up and red) or 1 (blue and down). We can specify the configuration of the entire system $\{\sigma\}$ with a list of all the local configurations.
  - Right now there aren't any *parameters* - in a "real" simulation, temperature could be one, but for now we just want to encode some physics in a simple matrix
  
So, this classical Hamiltonian says "sum up over all the connections on the square grid - neighbors having the same value is good, having different values is bad".

Note that there is *no* preferred direction/value - all being 0 (red/up) is just as good as all being 1 (blue/down).

Now we can write down some code! Right now our Hamiltonian just measures the energy of a given configuration - it is *diagonal* in the spin basis. So:

In [17]:
L = 10
function energy(config)
    E = 0.
    for i in 1:L^2
        # horizontal site
        j = mod(i, L) + L*div(i, L)
        if j > L^2 # literal corner case :)
            j = L^2 - L
        end
        E += -1 + 2*xor(config[i], config[j])
        # vertical site
        j = i + L
        if j > L^2
            j = mod(i + L, L^2) + 1
        end
        E += -1 + 2*xor(config[i], config[j])
    end
    return E
end

# make a matrix H
H = zeros(2^L, 2^L)
basis = Vector{Bool}[]

# translate the integer into a bit representation
for element in 1:2^L
    bit_rep = falses(L^2)
    for site in 1:L^2
       bit_rep[site] = (element >> (site - 1)) & 1
    end
    push!(basis, bit_rep)
end

We could have also used the Julia `digits` function to achieve the same outcome:

In [5]:
Vector{Bool}(digits(20, 2))

5-element Array{Bool,1}:
 false
 false
  true
 false
  true

The point is, we're going to use whether the bit is `0` or `1` on each "site" (site `i` is indexed as bit `i` in the integer) to encode the "state" of up/down on each site, then make a matrix describing the energy of each state. We'll deal with small systems for now, so whatever method is clearest to you is completely fine to use! First, let's check we did this correctly (each element of `basis` should be unique).

In [9]:
@assert length(unique(basis)) == 2^L "Basis elements aren't unique!"

In [19]:
for (index, configuration) in enumerate(basis)
    H[index, index] = energy(configuration)
end

In [20]:
@elapsed eig(H)

0.733284062

In [21]:
@elapsed eig(Diagonal(H))

0.011597495

Since this model is completely diagonal in the *simulation basis* (of lists of up/downs) it's pretty boring. Time to add some complications to it.